Testing a Semi-Definite Program.

A semidefinite program (SDP) is an optimization problem of the form
$$
\text{minimize   }  tr(CX) \\
\text{subject to  } tr(A_iX) = b_i, i = 1,...,p \\
             X >= 0, \\
$$

In [25]:
# Import packages.
import cvxpy as cp 
import numpy as np
import matlab

In [26]:
# Find all classical strategies for d

d = 2
# There will be d + 1 projections.

# There are 3 possible inputs per party and 2 possible outputs.
# inputs = {0, 1, 2} and outputs = {0, 1}
# 2^3 possible strategy per party, so 2^3 * 2^3 = 64 possible strategies
# Due to symmetricity of simplex vectors, e.g. (1, 0, 0) = (0, 1, 0) = (0, 0, 1)
# there will be 16 unique strategies


def find_strats(x, n):
    if n == 0:
        return x
    
    new = []
    for partial_strat in x:
        new.append(partial_strat + [0])
        new.append(partial_strat + [1])
    
    if not new:
        new = [[0], [1]]
    
    return find_strats(new, n - 1)

def generate_Pli(A, B):
    
    local_strat = np.zeros((2 * (d + 1), 2 * (d + 1)))
    
    for X in range(d + 1):
        for Y in range(d + 1):
            a = A[X]
            b = B[Y]
            
            local_strat[2 * X + a][2 * Y + b] = 1
    
    return local_strat

def generate_classical_strats():
    result = []
    
    for A in find_strats([], d + 1):
        for B in find_strats([], d + 1):
            result.append(generate_Pli(A, B))
    
    return result


min tr((M^T)p_q) 

subject to tr((M^T)P^{(L,i)}) >= 1 for all i

            M >= 0

In [112]:

P_L = generate_classical_strats()

def optimize_coefficients(limit=-1):
        
    Q = np.zeros((2 * (d + 1), 2 * (d + 1)))

    for i in range(2 * (d + 1)):
        for j in range(2 * (d + 1)):
            a = i % 2
            b = j % 2
            x = i // 2
            y = j // 2
            
            if a == b == 0 and x == y:
                Q[i][j] = 1 / d
            elif a == b == 1 and x == y:
                Q[i][j] = 1 - (1/d)
            elif a == b == 0 and x != y:
                Q[i][j] = 1 / (d**3)
            elif (a + 1) % 2 == b and x != y:
                Q[i][j] = (d**2 - 1)/(d ** 3)
            elif a == b == 1 and x != y:
                Q[i][j] = (d**3 - 2*(d**2) + 1)/(d ** 3)
            else:
                Q[i][j] = 0.0

    # for row in Q:
    #     print(row)

    M = cp.Variable((2 * (d + 1), 2 * (d + 1)), symmetric=True)

    constraints = [M <= 1, 
        cp.trace(M) == 2]
    constraints += [
        cp.trace(M.T @ P_L[i]) <= 1 for i in range(len(P_L))
    ]
    
    # constraints += [
    #     cp.trace(M.T @ P_L[i]) >= 0 for i in range(len(P_L))
    # ]

    if limit != -1:
        constraints += [
            cp.trace(M.T @ P_L[i]) >= limit
        ]
    
    prob = cp.Problem(cp.Maximize(cp.trace(M.T @ Q)), constraints)
    prob.solve()

    print("The optimal value is", prob.value)
    # print("A solution X is")
    # print(M.value)
    # print(cp.trace(M.T.value @ Q).value)
    return M.value
    
    # reshaped_array = M.value.reshape(3, 2, 3, 2)

    # # Swap axes to get the final shape 2x2x3x3
    # final_array = reshaped_array.swapaxes(1, 2)
    
    # return final_array
    # for x in final_array:
    #     for y in x:
    #         for t in y:
    #             print(y)
            # print(y)
    # return M.
    
    # coefficients = np.ones((2, 2, d + 1, d + 1))
    
    # for i
    # return coefficients

val = optimize_coefficients()
matlab_array = matlab.double(val)
# print(matlab_array)

# Write the desc 

The optimal value is 2.4999999985625827


In [113]:
import matlab.engine
eng = matlab.engine.start_matlab()

eng.addpath(r'/Users/jamkabeeralikhan/Documents/MATLAB')

def bellMax(d, arr):

    result = eng.Optimizer(d, matlab_array)

    # Print the result
    print(f"The result is: {result}")

print(bellMax(d, matlab_array))

# Close the MATLAB engine
eng.quit()


(:,:,1,1) =

    0.3333   -1.9464
   -1.9464    0.3333


(:,:,2,1) =

   -2.0000    1.0000
    1.0000   -2.0000


(:,:,3,1) =

   -2.0000    1.0000
    1.0000   -2.0000


(:,:,1,2) =

   -2.0000    1.0000
    1.0000   -2.0000


(:,:,2,2) =

    0.3333   -1.9464
   -1.9464    0.3333


(:,:,3,2) =

   -2.0000    1.0000
    1.0000   -2.0000


(:,:,1,3) =

   -2.0000    1.0000
    1.0000   -2.0000


(:,:,2,3) =

   -2.0000    1.0000
    1.0000   -2.0000


(:,:,3,3) =

    0.3333   -1.9464
   -1.9464    0.3333

The result is: 2.499999991440908
None
